# Enrichissement Hybride LLM - Seine-Saint-Denis Appels à Projets

Ce notebook scrape ET enrichit les données avec Claude Sonnet 4.5 pour extraire:
- **Scraping complet** du site seine-saint-denis.gouv.fr (avec pagination)
- **Filtrage** sur année N-1 (2025) et année N (2026)
- **Résumés structurés** via LLM
- **Montants (min/max)** extraits automatiquement
- **Catégories et tags** intelligents
- **Public cible** identifié
- **Modalités et démarches** structurées
- **Extraction PDF** des règlements

**Approche:** Workflow complet et indépendant = Scraping + LLM Claude pour enrichissement

## 1. Imports et configuration

In [ ]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import json
from urllib.parse import urljoin
import time
import os
from dotenv import load_dotenv
import tempfile
import hashlib
import itertools

# Imports LLM
from anthropic import Anthropic
import pypdf

In [ ]:
# Charger les variables d'environnement
load_dotenv(override=True)

# Vérifier Claude API key
claude_api_key = os.getenv('ANTHROPIC_API_KEY')
if claude_api_key:
    print(f"✅ ANTHROPIC_API_KEY trouvée: {claude_api_key[:10]}...")
else:
    print(f"❌ ANTHROPIC_API_KEY non trouvée dans .env")
    print(f"   ⚠️ Vous devez ajouter: ANTHROPIC_API_KEY=sk-ant-xxxxxx")

## 2. Configuration scraper

In [ ]:
# Configuration du scraper
BASE_URL = "https://www.seine-saint-denis.gouv.fr/Actualites/Appels-a-projets"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# Filtrage sur année N-1 et année N
CURRENT_YEAR = datetime.now().year
YEARS_TO_KEEP = [CURRENT_YEAR - 1, CURRENT_YEAR]

print(f"✅ Configuration prête")
print(f"   Base URL: {BASE_URL}")
print(f"   Années filtrées: {YEARS_TO_KEEP}")
print(f"   Dépendances: requests, BeautifulSoup4, pandas, anthropic, pypdf")

## 3. Scraper les appels à projets de Seine-Saint-Denis

In [ ]:
def fetch_all_pages(base_url, max_pages=5):
    """Récupérer toutes les pages avec pagination (offset)"""
    all_html_pages = []
    
    for page_num in range(max_pages):
        offset = page_num * 10
        if offset == 0:
            url = base_url
        else:
            url = f"{base_url}/(offset)/{offset}"
        
        print(f"🔄 Fetching page {page_num + 1} (offset={offset})...")
        
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Vérifier s'il y a du contenu (articles ou liens)
            articles = soup.find_all(['div', 'li', 'article'], class_=re.compile(r'(item|article|news|appel)', re.I))
            links = soup.find_all('a', href=re.compile(r'Appels-a-projets/.+', re.I))
            
            if not articles and not links:
                print(f"   ⚠️ Pas de contenu pertinent trouvé, arrêt pagination")
                break
            
            all_html_pages.append(response.text)
            print(f"   ✅ Page récupérée ({len(response.text)} chars)")
            
            time.sleep(1)  # Pause pour être poli
            
        except Exception as e:
            print(f"   ❌ Erreur: {str(e)[:50]}")
            break
    
    return all_html_pages

# Récupérer les pages (on limite à 5 pages pour l'exemple, augmenter si nécessaire)
html_pages = fetch_all_pages(BASE_URL, max_pages=5)
print(f"\n✅ {len(html_pages)} pages récupérées")

In [ ]:
def extract_years_from_text(text):
    """Extraire les années mentionnées dans un texte"""
    if not text:
        return []
    # Chercher 2024, 2025, 2026 etc.
    return [int(y) for y in re.findall(r'\b(202[0-9])\b', text)]

def scrape_ssd_aap(html_pages, years_filter=None):
    """Scrape les AAP du site Seine-Saint-Denis avec filtrage par année"""
    aap_list = []
    seen_urls = set()
    
    if not html_pages:
        return aap_list
    
    for page_html in html_pages:
        soup = BeautifulSoup(page_html, 'html.parser')
        
        # Stratégie 1: Chercher les blocs d'articles
        items = soup.find_all(['div', 'article'], class_=re.compile(r'(item|article|news-item)', re.I))
        
        # Stratégie 2: Si pas d'items clairs, chercher tous les liens dans la zone de contenu principal
        if not items:
            main_content = soup.find('div', id=re.compile(r'(main|content|centre)', re.I))
            if main_content:
                items = main_content.find_all('a', href=re.compile(r'Appels-a-projets/', re.I))
        
        for item in items:
            try:
                aap = {}
                
                # Extraction Titre et URL
                if item.name == 'a':
                    title_elem = item
                    link_elem = item
                    container = item.parent
                else:
                    title_elem = item.find(['h2', 'h3', 'h4', 'a'])
                    link_elem = item.find('a', href=True)
                    container = item
                
                if not title_elem or not link_elem:
                    continue
                    
                aap['titre'] = title_elem.get_text(strip=True)
                aap['url_source'] = urljoin(BASE_URL, link_elem.get('href', ''))
                
                # Ignorer si ce n'est pas un lien vers un AAP (ex: lien de pagination)
                if 'offset' in aap['url_source'] or aap['url_source'] == BASE_URL:
                    continue
                
                # Éviter les doublons
                if aap['url_source'] in seen_urls:
                    continue
                seen_urls.add(aap['url_source'])
                
                # Extraction du texte pour filtrage et résumé
                text_content = container.get_text(' ')
                
                # Filtrage par année
                if years_filter:
                    years_found = extract_years_from_text(text_content) + extract_years_from_text(aap['titre'])
                    # Si on trouve des années, on vérifie si l'une d'elles est dans notre filtre
                    # Si aucune année trouvée, on garde par défaut (le LLM vérifiera)
                    if years_found:
                        if not any(y in years_filter for y in years_found):
                            continue
                
                # Résumé préliminaire
                desc_elem = container.find(['p', 'div'], class_=re.compile(r'(desc|intro|chapo)', re.I))
                aap['resume'] = desc_elem.get_text(strip=True) if desc_elem else None
                
                # Dates (tentative d'extraction regex)
                dates = re.findall(r'\d{1,2}[/\-]\d{1,2}[/\-]\d{4}', text_content)
                if dates:
                    try:
                        # On prend la dernière date comme date limite potentielle
                        aap['date_limite'] = pd.to_datetime(dates[-1].replace('-', '/'), dayfirst=True).date()
                    except:
                        aap['date_limite'] = None
                else:
                    aap['date_limite'] = None
                
                # Champs par défaut
                aap['organisme'] = 'Préfecture de Seine-Saint-Denis'
                aap['perimetre_geo'] = 'Seine-Saint-Denis (93)'
                aap['id_record'] = f"ssd_{datetime.now().strftime('%Y%m%d')}_{hash(aap['url_source']) % 100000}"
                
                aap_list.append(aap)
                print(f"   ✅ Trouvé: {aap['titre'][:60]}...")
                
            except Exception as e:
                print(f"   ⚠️ Erreur parsing item: {str(e)}")
                continue
    
    return aap_list

# Exécuter le scraping
aap_data = scrape_ssd_aap(html_pages, years_filter=YEARS_TO_KEEP)
print(f"\n✅ {len(aap_data)} appels à projets retenus (Filtre années: {YEARS_TO_KEEP})")

## 4. Créer et nettoyer le DataFrame

In [ ]:
# Créer DataFrame
if aap_data:
    mapped_df_ssd = pd.DataFrame(aap_data)
    print(f"📊 DataFrame créé: {mapped_df_ssd.shape}")
else:
    mapped_df_ssd = pd.DataFrame(columns=['titre', 'url_source', 'resume', 'date_limite', 'organisme'])
    print("⚠️ Aucune donnée trouvée")

In [ ]:
# Fonction de nettoyage du texte
def clean_text(text):
    if not isinstance(text, str):
        return text
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Appliquer le nettoyage
if not mapped_df_ssd.empty:
    for col in mapped_df_ssd.select_dtypes(include=['object']).columns:
        mapped_df_ssd[col] = mapped_df_ssd[col].apply(clean_text)

    # Ajouter colonnes manquantes pour l'enrichissement
    cols_to_add = ['montant_max', 'montant_min', 'public_cible', 'categories', 'mots_cles', 'objectif', 'modalite', 'demarches', 'contact', 'taux_financement']
    for col in cols_to_add:
        if col not in mapped_df_ssd.columns:
            mapped_df_ssd[col] = None
            
    # Fingerprint unique
    if 'fingerprint' not in mapped_df_ssd.columns:
        mapped_df_ssd['fingerprint'] = mapped_df_ssd.apply(
            lambda row: hashlib.md5(f"{row.get('titre')}|{row.get('url_source')}".encode()).hexdigest()[:12], 
            axis=1
        )

    print("✅ DataFrame préparé pour l'enrichissement")
    print(f"   Colonnes: {list(mapped_df_ssd.columns)}")

## 5. Fonctions pour extraction PDF

In [ ]:
def extract_pdf_text(pdf_url, max_pages=3):
    """Extraire le texte d'un PDF depuis une URL"""
    try:
        response = requests.get(pdf_url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        
        with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as tmp:
            tmp.write(response.content)
            tmp_path = tmp.name
        
        reader = pypdf.PdfReader(tmp_path)
        text = ''
        for page_num, page in enumerate(reader.pages[:max_pages]):
            text += page.extract_text() + '\n'
        
        os.remove(tmp_path)
        return text if text.strip() else None
    except Exception as e:
        # print(f"  ⚠️ Erreur PDF {pdf_url.split('/')[-1]}: {str(e)[:30]}")
        return None

def find_pdf_links(soup, base_url):
    """Trouver les liens PDF dans une page"""
    pdf_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href', '')
        text = link.get_text().lower()
        
        if ('pdf' in href.lower() or 
            any(keyword in text for keyword in ['reglement', 'document', 'cahier', 'guide', 'télécharger'])):
            full_url = urljoin(base_url, href)
            if full_url not in pdf_links and full_url.lower().endswith('.pdf'):
                pdf_links.append(full_url)
    
    return pdf_links[:2]  # Limiter à 2 PDFs max pour ne pas surcharger

print("✅ Fonctions PDF créées")

## 6. Classe LLMEnricher

In [ ]:
class LLMEnricher:
    """Enrichir les données AAP avec Claude Sonnet 4.5"""
    
    def __init__(self, api_key=None, model='claude-sonnet-4-5'):
        self.api_key = api_key or os.getenv('ANTHROPIC_API_KEY')
        self.model = model
        self.client = Anthropic(api_key=self.api_key) if self.api_key else None
        self.max_retries = 3
        
        if not self.client:
            raise ValueError('❌ ANTHROPIC_API_KEY non trouvée')
    
    def extract_full_page(self, url, html_content, pdf_texts=None):
        """Extraire toutes les données manquantes d'une page"""
        
        if not self.client:
            return None
        
        soup = BeautifulSoup(html_content, 'html.parser')
        # Nettoyer le HTML pour réduire la taille
        for script in soup(["script", "style", "nav", "footer"]):
            script.decompose()
            
        text_content = soup.get_text('\n')
        text_content = re.sub(r'\n+', '\n', text_content).strip()
        
        if pdf_texts:
            text_content += '\n\n--- CONTENU DES DOCUMENTS PDF JOINTS ---\n'
            text_content += '\n\n'.join(pdf_texts)
        
        # Tronquer si trop long (Claude a une grande fenêtre mais restons raisonnables)
        text_content = text_content[:25000]
        
        prompt = f"""Tu es un expert en analyse d'appels à projets publics pour la Seine-Saint-Denis.
        
Analyse le texte ci-dessous (page web + potentiels PDFs) et extrais les informations structurées en JSON.

FORMAT JSON ATTENDU:
{{
  "resume": "Résumé synthétique en 2-3 phrases (max 400 caractères)",
  "montant_max": montant maximum en euros (nombre ou null), 
  "montant_min": montant minimum en euros (nombre ou null),
  "taux_financement": "pourcentage ou description courte (null si non trouvé)",
  "categories": ["liste", "de", "catégories", "pertinentes"],
  "public_cible": ["associations", "collectivités", "entreprises", "particuliers"],
  "mots_cles": ["3-5", "mots-clés", "importants"],
  "objectif": "Objectif principal de l'appel à projet",
  "modalite": "Conditions principales d'éligibilité",
  "demarches": "Comment candidater (plateforme, email, dossier)",
  "contact": "Email ou téléphone de contact (ou null)",
  "date_limite": "YYYY-MM-DD" (si trouvée dans le texte, sinon null)
}}

RÈGLES:
- Retourne UNIQUEMENT du JSON valide.
- Si une info n'existe pas, mets null.
- Sois précis sur les montants.
- Pour la date limite, essaie de trouver la date exacte de clôture.

TEXTE À ANALYSER:
{text_content}"""
        
        for attempt in range(self.max_retries):
            try:
                message = self.client.messages.create(
                    model=self.model,
                    max_tokens=1500,
                    messages=[{"role": "user", "content": prompt}]
                )
                
                response_text = message.content[0].text
                # Nettoyage du markdown json si présent
                response_text = response_text.replace('```json', '').replace('```', '').strip()
                
                return json.loads(response_text)
            except Exception as e:
                if attempt == self.max_retries - 1:
                    print(f"    ❌ Erreur LLM après {self.max_retries} essais: {str(e)[:50]}")
                    return None
                time.sleep(1)

print("✅ Classe LLMEnricher créée")

## 7. Aperçu des PDFs extraits (Test)

In [ ]:
if not mapped_df_ssd.empty:
    print("📄 Test extraction PDF sur le premier élément:")
    row = mapped_df_ssd.iloc[0]
    url = row['url_source']
    print(f"   URL: {url}")
    
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(resp.text, 'html.parser')
        pdfs = find_pdf_links(soup, url)
        print(f"   PDFs trouvés: {len(pdfs)}")
        for pdf in pdfs:
            print(f"   - {pdf.split('/')[-1]}")
    except Exception as e:
        print(f"   Erreur: {e}")

## 8. Initialiser l'enrichisseur LLM

In [ ]:
try:
    enricher = LLMEnricher()
    print("✅ LLMEnricher initialisé avec succès")
except ValueError as e:
    print(f"❌ {str(e)}")
    enricher = None

## 9. Enrichir avec LLM

In [ ]:
if enricher and not mapped_df_ssd.empty:
    print(f"🔄 Enrichissement de {len(mapped_df_ssd)} appels à projets...\n")
    
    for idx, row in mapped_df_ssd.iterrows():
        url = row.get('url_source')
        titre = str(row.get('titre', 'N/A'))[:50]
        
        print(f"🔄 [{idx+1}/{len(mapped_df_ssd)}] {titre}...", end=' ')
        
        try:
            # 1. Récupérer le contenu de la page
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            
            # 2. Chercher et extraire les PDFs
            soup = BeautifulSoup(response.text, 'html.parser')
            pdf_links = find_pdf_links(soup, url)
            
            pdf_texts = []
            if pdf_links:
                print(f"(+{len(pdf_links)} PDFs)", end=' ')
                for pdf_url in pdf_links:
                    pdf_text = extract_pdf_text(pdf_url)
                    if pdf_text:
                        pdf_texts.append(pdf_text[:4000]) # Limite par PDF
            
            # 3. Appel LLM
            extracted = enricher.extract_full_page(url, response.text, pdf_texts)
            
            # 4. Mise à jour du DataFrame
            if extracted:
                for key, value in extracted.items():
                    if key in mapped_df_ssd.columns:
                        # Si date_limite trouvée par LLM et vide dans le DF, on met à jour
                        if key == 'date_limite' and value:
                            mapped_df_ssd.at[idx, key] = value
                        # Pour les autres champs, on écrase ou remplit
                        elif value is not None:
                            mapped_df_ssd.at[idx, key] = value
                print("✅")
            else:
                print("⚠️  (Pas de réponse LLM)")
                
        except Exception as e:
            print(f"❌ {str(e)[:30]}")
        
        time.sleep(1) # Rate limiting
    
    print(f"\n✅ Enrichissement terminé!")
else:
    print("❌ Pas d'enrichissement (soit pas de données, soit pas de clé API)")

## 10. Statistiques et aperçu

In [ ]:
if not mapped_df_ssd.empty:
    print("📊 Statistiques de remplissage:")
    for col in ['montant_max', 'date_limite', 'public_cible', 'categories']:
        filled = mapped_df_ssd[col].notna().sum()
        print(f"   - {col}: {filled}/{len(mapped_df_ssd)} ({filled/len(mapped_df_ssd)*100:.1f}%)")
        
    print("\n📋 Aperçu des données enrichies:")
    from IPython.display import display, HTML
    display(mapped_df_ssd.head(3))

## 11. Exporter les données

In [ ]:
if not mapped_df_ssd.empty:
    # Export CSV
    csv_path = '../data/ssd_aap_enriched.csv'
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    mapped_df_ssd.to_csv(csv_path, index=False)
    print(f"✅ Données exportées vers {csv_path}")
    
    # Export JSON
    json_path = '../data/ssd_aap_enriched.json'
    mapped_df_ssd.to_json(json_path, orient='records', force_ascii=False, indent=2)
    print(f"✅ Données exportées vers {json_path}")

## 12. Upload Airtable (Optionnel)

In [ ]:
# if not mapped_df_ssd.empty:
#     from appels_a_projets.connectors.airtable_connector import AirtableConnector
#     connector = AirtableConnector()
#     connector.upload_dataframe(mapped_df_ssd)
#     print("✅ Upload vers Airtable terminé")